# Pricing Transition on Localnet

## Overview

The Arweave protocol transitions to the new dynamic (v2) pricing over
a multi-phased transition period. The final phase — the **2.7.2 transition** — linearly
interpolates between a cap price (340 Winston/GiB-minute) and the dynamically computed
v2 price over 518,400 blocks (~24 months).

This notebook validates pricing behavior around the **transition end** — the first block
where pure v2 pricing takes effect with no interpolation or bounds. The localnet snapshot
starts 5 blocks before the transition boundary.

**Sections:**
1. **Setup** – connect to the node, define RPC helpers and dollar-price conversions.
2. **Transition window** – query the transition parameters and confirm heights.
3. **Mine past transition** – mine ~40 blocks to cross the boundary and then trigger the first
   post-transition price adjustment.
4. **Fetch & display** – collect block pricing data and print a table with
   $\$/GiB$ upload costs (assuming $10/AR).
5. **Validate `is_v2_pricing_height`** – the flag transitions at the right height.
6. **Validate interpolation** – pre-transition target prices follow the interpolation formula.
7. **Validate V2 pricing** – post-transition target prices equal the raw v2 price.
8. **Validate continuity** – no sudden price jump at the boundary.
9. **Validate block field evolution** – `price_per_gib_minute` and
   `scheduled_price_per_gib_minute` evolve per the EMA recalculation rule.

## Setup

### Connect to the localnet node

Starts a distributed Erlang node with long names, sets the cookie to `localnet`,
and pings `main-localnet@127.0.0.1` to confirm connectivity.

In [ ]:
Cookie = 'localnet',
Node = 'main-localnet@127.0.0.1',

HostHasDot =
	case string:split(atom_to_list(node()), "@") of
		[_Name, Host] ->
			case string:find(Host, ".") of
				nomatch ->
					false;
				_ ->
					true
			end;
		_ ->
			false
	end,

_ =
	case {node(), HostHasDot} of
		{nonode@nohost, _} ->
			net_kernel:start([list_to_atom("pricing_notebook@127.0.0.1"), longnames]);
		{_, true} ->
			ok;
		{_, false} ->
			net_kernel:stop(),
			net_kernel:start([list_to_atom("pricing_notebook@127.0.0.1"), longnames])
	end,

true = erlang:set_cookie(node(), Cookie),
ok.

In [ ]:
{Node, pong} = {Node, net_adm:ping(Node)},
ok.

### RPC and mining helpers

- `RPCCall(M, F, A)` – calls `M:F(A)` on the remote node (30 s timeout).
- `RPCHeight()` – current block height.
- `MineUntilHeight(H)` – asks localnet to mine to height H and polls until reached.
- `RPCBlockByHeight(H)` – reads the block record at height H.
- `RPCGetPricePerGiBMinute(H, Block)` – `ar_pricing:get_price_per_gib_minute/2`.
- `RPCGetV2PricePerGiBMinute(H, Block)` – `ar_pricing:get_v2_price_per_gib_minute/2`.
- `RPCIsV2PricingHeight(H)` – `ar_pricing_transition:is_v2_pricing_height/1`.
- `RPCGetTxFee(Size, Price, Kryder, H)` – `ar_pricing:get_tx_fee/1`.

In [ ]:
RPCCall = fun(M, F, A) -> rpc:call(Node, M, F, A, 30000) end,
RPCHeight = fun() -> RPCCall(ar_node, get_height, []) end,

WaitForHeight =
	fun
		(_, _TargetHeight, 0) ->
			error(mine_until_height_timeout);
		(Self, TargetHeight, AttemptsLeft) ->
			case RPCHeight() >= TargetHeight of
				true ->
					ok;
				false ->
					timer:sleep(100),
					Self(Self, TargetHeight, AttemptsLeft - 1)
			end
	end,

MineUntilHeight =
	fun(TargetHeight) ->
		MineResult = RPCCall(ar_localnet, mine_until_height, [TargetHeight]),
		ok =
			case MineResult of
				ok ->
					ok;
				[] ->
					ok;
				Other ->
					error({unexpected_mine_until_height_result, Other})
			end,
		WaitForHeight(WaitForHeight, TargetHeight, 6000)
	end,

RPCBlockHashByHeight =
	fun(H) ->
		RPCCall(ar_block_index, get_element_by_height, [H])
	end,

RPCBlockByHeight =
	fun(H) ->
		Hash = RPCBlockHashByHeight(H),
		RPCCall(ar_storage, read_block, [Hash])
	end,

RPCIsV2PricingHeight =
	fun(H) ->
		RPCCall(ar_pricing_transition, is_v2_pricing_height, [H])
	end,

RPCGetTxFee =
	fun(DataSize, Price, Kryder, H) ->
		RPCCall(ar_pricing, get_tx_fee, [{DataSize, Price, Kryder, H}])
	end,

ok.

### Record accessors and dollar-price helpers

Compiles `nb_block` at runtime for record field access. Defines HTTP helpers
and dollar conversion assuming **$10/AR**:

- `WinstonToUSD(W, D)` – converts Winston to USD at denomination `D`.
  Formula: $$W × $10 / (10^{12} × 1000^{D−1})$$
- `UploadCostUSD(Price, Kryder, H, D)` – the cost to upload 1 GiB in USD.
  Uses `ar_pricing:get_tx_fee/1` which accounts for perpetual storage
  (200+ years with 0.5 %/year decay), 20 replicas, Kryder+ rate, and the 5 % miner share.

In [ ]:
TmpDir = ".tmp/notebooks/",
ok = filelib:ensure_dir(filename:join([TmpDir, "keep"])),

CompileModule =
	fun(Name, Source) ->
		Path = filename:join([TmpDir, Name ++ ".erl"]),
		ok = file:write_file(Path, Source),
		{ok, Module, Bin} = compile:file(Path, [binary]),
		{module, Module} = code:load_binary(Module, Path, Bin)
	end,

BlockAccessors =
	lists:flatten([
		"-module(nb_block).\n",
		"-export([height/1, price_per_gib_minute/1, scheduled_price_per_gib_minute/1,\n",
		"         denomination/1, kryder_plus_rate_multiplier/1]).\n",
		"-include_lib(\"arweave/include/ar.hrl\").\n",
		"height(B) -> B#block.height.\n",
		"price_per_gib_minute(B) -> B#block.price_per_gib_minute.\n",
		"scheduled_price_per_gib_minute(B) -> B#block.scheduled_price_per_gib_minute.\n",
		"denomination(B) -> B#block.denomination.\n",
		"kryder_plus_rate_multiplier(B) -> B#block.kryder_plus_rate_multiplier.\n"
	]),

CompileModule("nb_block", BlockAccessors),

%% Remote helper: runs on the node to avoid sending block records over RPC.
%% get_target_and_v2/1 reads the previous block from local storage
%% and computes both the target price and the v2 price on-node.
RemotePricingHelper =
	lists:flatten([
		"-module(nb_remote_pricing).\n",
		"-export([get_target_and_v2/1]).\n",
		"-include_lib(\"arweave/include/ar.hrl\").\n",
		"get_target_and_v2(Height) ->\n",
		"    PrevHash = element(1, ar_block_index:get_element_by_height(Height - 1)),\n",
		"    PrevBlock = ar_block_cache:get(block_cache, PrevHash),\n",
		"    case PrevBlock of\n",
		"        not_found ->\n",
		"            {error, error};\n",
		"        _ ->\n",
		"            V2 = try ar_pricing:get_v2_price_per_gib_minute(Height, PrevBlock)\n",
		"                 catch _:_ -> error end,\n",
		"            Target = try ar_pricing:get_price_per_gib_minute(Height, PrevBlock)\n",
		"                    catch _:_ -> error end,\n",
		"            {Target, V2}\n",
		"    end.\n"
	]),
RemotePricingPath = filename:join([TmpDir, "nb_remote_pricing.erl"]),
ok = file:write_file(RemotePricingPath, RemotePricingHelper),
{ok, nb_remote_pricing, RemotePricingBin} = compile:file(RemotePricingPath, [binary]),
{module, nb_remote_pricing} = code:load_binary(nb_remote_pricing, RemotePricingPath, RemotePricingBin),
%% Load on the remote node
{module, nb_remote_pricing} =
	RPCCall(code, load_binary, [nb_remote_pricing, RemotePricingPath, RemotePricingBin]),

RPCGetTargetAndV2 =
	fun(H) ->
		RPCCall(nb_remote_pricing, get_target_and_v2, [H])
	end,

{ok, _} = application:ensure_all_started(inets),

LocalnetHTTPHost =
	case os:getenv("LOCALNET_HTTP_HOST") of
		false ->
			"127.0.0.1";
		V ->
			V
	end,

LocalnetHTTPPort =
	case os:getenv("LOCALNET_HTTP_PORT") of
		false ->
			"1984";
		V ->
			V
	end,

LocalnetNetworkName =
	case os:getenv("LOCALNET_NETWORK_NAME") of
		false ->
			"arweave.localnet";
		V ->
			V
	end,

BaseUrl = "http://" ++ LocalnetHTTPHost ++ ":" ++ LocalnetHTTPPort,

HTTPGet =
	fun(Url) ->
		Headers = [{"x-network", LocalnetNetworkName}],
		case httpc:request(get, {Url, Headers}, [], []) of
			{ok, {{_, 200, _}, _, Body}} ->
				{ok, Body};
			{ok, {{_, Status, _}, _, Body}} ->
				{error, {http_status, Status, Body}};
			Error ->
				{error, Error}
		end
	end,

HTTPGetInteger =
	fun(Url) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				binary_to_integer(iolist_to_binary(Body));
			{error, Reason} ->
				{error, Reason}
		end
	end,

GiB = 1073741824,

WinstonToUSD =
	fun(Winston, Denom) ->
		Pow = lists:foldl(fun(_, Acc) -> Acc * 1000 end, 1, lists:seq(1, Denom - 1)),
		Winston * 10.0 / (1000000000000 * Pow)
	end,

UploadCostUSD =
	fun(Price, Kryder, H, Denom) ->
		TxFee = RPCGetTxFee(GiB, Price, Kryder, H),
		WinstonToUSD(TxFee, Denom)
	end,

ok.

## Transition Window

Queries the 2.7.2 pricing transition parameters via RPC.

The 2.7.2 transition linearly interpolates between a **start price** (the 2.7.2 cap =
340 Winston/GiB-minute) and the dynamic v2 price. The start price is obtained by
evaluating `get_transition_price(TransitionStart, 0)`: at the transition start all weight
is on the start price, so passing a v2 price of 0 yields the start price exactly.

**Queried values:**
- `ar_pricing_transition:transition_start_2_7_2()` – first height of the 2.7.2 transition.
- `ar_pricing_transition:transition_length_2_7_2()` – number of transition blocks.
- Transition end = start + length (first block with pure v2 pricing).
- `PRICE_ADJUSTMENT_FREQUENCY` = 50 blocks (production/localnet value).

In [ ]:
TransitionStart = RPCCall(ar_pricing_transition, transition_start_2_7_2, []),
TransitionLength = RPCCall(ar_pricing_transition, transition_length_2_7_2, []),
TransitionEnd = TransitionStart + TransitionLength,
Height0 = RPCHeight(),

TransitionStartPrice =
	RPCCall(ar_pricing_transition, get_transition_price, [TransitionStart, 0]),

PriceAdjustFreq = 50,
FirstPostAdjust =
	case TransitionEnd rem PriceAdjustFreq of
		0 ->
			TransitionEnd;
		_ ->
			((TransitionEnd div PriceAdjustFreq) + 1) * PriceAdjustFreq
	end,

true = is_integer(TransitionStart),
true = is_integer(TransitionLength),
true = is_integer(TransitionEnd),
true = (TransitionLength > 0),
true = (TransitionEnd == TransitionStart + TransitionLength),
340 = TransitionStartPrice,
SnapshotBlocksBeforeEnd = TransitionEnd - Height0,
5 = SnapshotBlocksBeforeEnd,

io:format("  Transition start (2.7.2):       ~p~n", [TransitionStart]),
io:format("  Transition length:              ~p blocks~n", [TransitionLength]),
io:format("  Transition end:                 ~p~n", [TransitionEnd]),
io:format("  Transition start price:         ~p Winston/GiB-min~n", [TransitionStartPrice]),
io:format("  Current height:                 ~p~n", [Height0]),
io:format("  Blocks until transition end:    ~p~n", [SnapshotBlocksBeforeEnd]),
io:format("  Price adjustment frequency:     ~p blocks~n", [PriceAdjustFreq]),
io:format("  First post-transition adjust:   ~p~n", [FirstPostAdjust]),
ok.

## Mine Past Transition End

The snapshot starts 5 blocks before the transition end. We mine past the boundary and
past the first post-transition price-adjustment height so we can validate the EMA
recalculation under pure v2 pricing.

**Mined blocks:** from the current height to `FirstPostAdjust + 5` (~40 blocks).
**Submitted txs:** none.
**Expected:** mining succeeds without errors through the transition boundary.

In [ ]:
MineTarget = FirstPostAdjust + 5,
io:format("  Mining from ~p to ~p (~p blocks)...~n",
	[Height0, MineTarget, MineTarget - Height0]),
ok = MineUntilHeight(MineTarget),
HeightAfterMine = RPCHeight(),
true = (HeightAfterMine >= MineTarget),
io:format("  Done. Current height: ~p~n", [HeightAfterMine]),
ok.

## Fetch and Display Pricing Data

Fetches block records from `TransitionEnd - 5` to `MineTarget` and builds a table
with the following columns:

| Column | Source |
|--------|--------|
| **Height** | block height |
| **Price** | `block.price_per_gib_minute` (stored, EMA-smoothed; updates every 50 blocks) |
| **Scheduled Price** | `block.scheduled_price_per_gib_minute` (next value for Price at the next adjustment) |
| **Target** | `ar_pricing:get_price_per_gib_minute(H, PrevBlock)` — the price at the given height adjusted for the transition |
| **V2 Price** | `ar_pricing:get_v2_price_per_gib_minute(H, PrevBlock)` — raw dynamic price (no transition) |
| **V2?** | `ar_pricing_transition:is_v2_pricing_height(H)` |
| $\mathbf{\$/GiB}$| Upload cost for 1 GiB in USD at $\$10/AR$ (via `get_tx_fee`, includes decay, 20 replicas, miner share) |

Why `price_per_gib_minute` can look small: it is a **Winston per GiB-minute unit rate**, not a direct upload fee. The upload fee path annualizes this rate and applies replication, Kryder+, and miner-share factors in `get_tx_fee/1`, so a single-digit GiB-minute rate can still produce a meaningful `$ / GiB` upload cost.

In [ ]:
RangeStart = TransitionEnd - 5,
RangeEnd = MineTarget,
true = (RangeStart =< RangeEnd),

Blocks = maps:from_list(
	[{H, RPCBlockByHeight(H)} || H <- lists:seq(RangeStart, RangeEnd)]
),

Heights = lists:seq(RangeStart, RangeEnd),
true = (maps:size(Blocks) == length(Heights)),

Rows = lists:map(
	fun(H) ->
		Block = maps:get(H, Blocks),
		Price = nb_block:price_per_gib_minute(Block),
		Scheduled = nb_block:scheduled_price_per_gib_minute(Block),
		Denom = nb_block:denomination(Block),
		Kryder = nb_block:kryder_plus_rate_multiplier(Block),
		IsV2 = RPCIsV2PricingHeight(H),
		{TargetPrice, V2Price} = RPCGetTargetAndV2(H),
		USD = UploadCostUSD(Price, Kryder, H, Denom),
		#{ height => H, price => Price, scheduled => Scheduled,
		  denomination => Denom, kryder => Kryder,
		  is_v2 => IsV2, v2_price => V2Price, target => TargetPrice,
		  upload_usd => USD }
	end,
	Heights),

TargetErrors = [maps:get(height, R) || R <- Rows, maps:get(target, R) == error],
V2Errors = [maps:get(height, R) || R <- Rows, maps:get(v2_price, R) == error],
[] = TargetErrors,
[] = V2Errors,

io:format("~n~10s | ~6s | ~6s | ~7s | ~7s | ~3s | ~s~n",
	["Height", "Price", "Sched", "Target", "V2", "V2?", "$/GiB upload"]),
io:format("~s~n", [lists:duplicate(72, $-)]),

lists:foreach(
	fun(Row) ->
		#{height := H, price := P, scheduled := S, target := T,
		  v2_price := V2, is_v2 := IV2, upload_usd := U} = Row,
		V2Flag =
			case IV2 of
				true ->
					"yes";
				false ->
					"no "
			end,
		Mark =
			case H of
				_ when H == TransitionEnd ->
					"  <-- transition end";
				_ when H == FirstPostAdjust ->
					"  <-- 1st post-adjust";
				_ ->
					""
			end,
		FmtInt =
			fun(error) -> "err";
			   (N) -> integer_to_list(N)
			end,
		UStr = lists:flatten(io_lib:format("$~.4f", [U])),
		io:format("~10s | ~6s | ~6s | ~7s | ~7s | ~3s | ~s~s~n",
			[integer_to_list(H), integer_to_list(P), integer_to_list(S),
			 FmtInt(T), FmtInt(V2), V2Flag, UStr, Mark])
	end,
	Rows),

FirstRow = hd(Rows),
Price0 = maps:get(price, FirstRow),
Kryder0 = maps:get(kryder, FirstRow),
HeightPrice0 = maps:get(height, FirstRow),
Denom0 = maps:get(denomination, FirstRow),
UploadWinston0 = RPCGetTxFee(GiB, Price0, Kryder0, HeightPrice0),
UploadUSD0 = UploadCostUSD(Price0, Kryder0, HeightPrice0, Denom0),
io:format("~n  Sample at height ~p: price_per_gib_minute=~p, fee_1GiB=~p Winston (~.4f USD)~n",
	[HeightPrice0, Price0, UploadWinston0, UploadUSD0]),
true = (UploadWinston0 > 0),

ok.

## Validate `is_v2_pricing_height`

`is_v2_pricing_height(H)` must be `false` for all `H < TransitionEnd`
and `true` for all `H >= TransitionEnd`.

**Expected:** the flag transitions exactly at `TransitionEnd`.
**Queried:** `ar_pricing_transition:is_v2_pricing_height/1` via RPC (already fetched in `Rows`).

In [ ]:
PreV2Rows = [R || R <- Rows, maps:get(height, R) < TransitionEnd],
PostV2Rows = [R || R <- Rows, maps:get(height, R) >= TransitionEnd],
true = (length(PreV2Rows) > 0),
true = (length(PostV2Rows) > 0),

true = lists:all(
	fun(R) -> maps:get(is_v2, R) == false end,
	PreV2Rows
),

true = lists:all(
	fun(R) -> maps:get(is_v2, R) == true end,
	PostV2Rows
),

io:format("  All pre-transition heights:  is_v2 = false  [OK]~n"),
io:format("  All post-transition heights: is_v2 = true   [OK]~n"),
ok.

## Validate Pre-Transition Interpolation

For every height `H < TransitionEnd`, the target price returned by
`ar_pricing:get_price_per_gib_minute(H, PrevBlock)` must match the interpolation
formula used inside `get_transition_price`:

```
Interval1 = H - TransitionStart
Interval2 = TransitionEnd - H
Expected  = (StartPrice * Interval2 + V2Price * Interval1) div (Interval1 + Interval2)
```

where `StartPrice` = 340 (the 2.7.2 cap) and `V2Price = get_v2_price_per_gib_minute(H, PrevBlock)`.

During the 2.7.2 transition, bounds are `[0, infinity)`, so the `between` clamp is a no-op.

Expectation note: this catches interpolation/transition arithmetic issues in
`get_price_per_gib_minute/2`, but still depends on `get_v2_price_per_gib_minute/2`
for the V2 component.

In [ ]:
PreRows = [R || R <- Rows, maps:get(height, R) < TransitionEnd],
PreRowsValid = [R || R <- PreRows,
	maps:get(target, R) /= error, maps:get(v2_price, R) /= error],
true = (length(PreRows) > 0),
true = (length(PreRowsValid) > 0),

lists:foreach(
	fun(Row) ->
		H = maps:get(height, Row),
		V2 = maps:get(v2_price, Row),
		Target = maps:get(target, Row),
		Interval1 = H - TransitionStart,
		Interval2 = TransitionEnd - H,
		Expected = (TransitionStartPrice * Interval2 + V2 * Interval1)
			div (Interval1 + Interval2),
		case Target == Expected of
			true ->
				ok;
			false ->
				error({interpolation_mismatch, H, Target, Expected})
		end
	end,
	PreRowsValid),

io:format("  ~p/~p pre-transition heights match interpolation formula  [OK]~n",
	[length(PreRowsValid), length(PreRows)]),
ok.

## Validate Post-Transition V2 Pricing

For every height `H >= TransitionEnd`, the transition is complete and
`get_price_per_gib_minute(H, PrevBlock)` must equal
`get_v2_price_per_gib_minute(H, PrevBlock)` — no interpolation, no bounds.

Expectation note: this directly validates the transition handoff, but if both
functions shared the same V2 defect it would not catch that defect by itself.

In [ ]:
PostRows = [R || R <- Rows, maps:get(height, R) >= TransitionEnd],
PostRowsValid = [R || R <- PostRows,
	maps:get(target, R) /= error, maps:get(v2_price, R) /= error],
true = (length(PostRows) > 0),
true = (length(PostRowsValid) > 0),

lists:foreach(
	fun(Row) ->
		H = maps:get(height, Row),
		V2 = maps:get(v2_price, Row),
		Target = maps:get(target, Row),
		case Target == V2 of
			true ->
				ok;
			false ->
				error({v2_price_mismatch, H, Target, V2})
		end
	end,
	PostRowsValid),

io:format("  ~p/~p post-transition heights: target == V2 price  [OK]~n",
	[length(PostRowsValid), length(PostRows)]),
ok.

## Validate Price Continuity at Transition Boundary

At `TransitionEnd - 1` (last interpolated block), almost all weight is on V2Price:

```
weight_on_V2 = (TransitionLength - 1) / TransitionLength ~ 0.999998
```

The gap between the last interpolated price and the first pure-V2 price is at most
`|StartPrice - V2Price| / TransitionLength`, which for `TransitionLength = 518,400`
is negligible.

**Expected:** relative price change < 0.1 % (1e-3).
**Queried:** target prices from `Rows`.

In [ ]:
LastPreValid = [R || R <- PreRows, maps:get(target, R) /= error],
FirstPostValid = [R || R <- PostRows, maps:get(target, R) /= error],
true = (length(LastPreValid) > 0),
true = (length(FirstPostValid) > 0),

LastPreTarget = maps:get(target, lists:last(LastPreValid)),
FirstPostTarget = maps:get(target, hd(FirstPostValid)),
Gap = abs(FirstPostTarget - LastPreTarget),
RelGap = Gap / max(1, LastPreTarget),
io:format("  Last pre-transition target:   ~p~n", [LastPreTarget]),
io:format("  First post-transition target:  ~p~n", [FirstPostTarget]),
io:format("  Absolute gap:                  ~p~n", [Gap]),
io:format("  Relative gap:                  ~.8f~n", [RelGap]),
true = RelGap < 0.001,
io:format("  Relative gap < 0.1 %%  [OK]~n"),
ok.

## Validate Block Price Field Evolution

The block's `price_per_gib_minute` field (stored on-chain) updates only at
**price adjustment heights** (`Height rem 50 == 0`). The recalculation rule
(post fork 2.7.1) is:

```
NewPrice     = PrevBlock.scheduled_price_per_gib_minute
TargetPrice  = get_price_per_gib_minute(Height, PrevBlock)
EMAPrice     = (9 * PrevScheduled + TargetPrice) div 10
NewScheduled = max(PrevScheduled div 2, min(PrevScheduled * 2, EMAPrice))
```

Between adjustments both fields are unchanged.

The nearest adjustment height **after** the transition end is **`FirstPostAdjust`**
(= `TransitionEnd` rounded up to the next multiple of 50). At that height the
target price is a pure v2 price for the first time — this verifies the
recalculation correctly handles the transition boundary.

Expectation note: this cell derives `ExpectedTargetPrice` from transition math +
`v2_price`, then checks EMA against block fields, instead of reusing the already
queried `target` value for the EMA expectation.

In [ ]:
Pairs = lists:zip(lists:droplast(Rows), tl(Rows)),
true = (length(Pairs) > 0),

lists:foreach(
	fun({PrevRow, CurrRow}) ->
		H = maps:get(height, CurrRow),
		Price = maps:get(price, CurrRow),
		Sched = maps:get(scheduled, CurrRow),
		PrevPrice = maps:get(price, PrevRow),
		PrevSched = maps:get(scheduled, PrevRow),
		IsAdjust = (H rem PriceAdjustFreq == 0),
		case IsAdjust of
			false ->
				case {Price == PrevPrice, Sched == PrevSched} of
					{true, true} ->
						ok;
					_ ->
						error({unexpected_price_change, H,
							{price, Price, PrevPrice},
							{sched, Sched, PrevSched}})
				end;
			true ->
				case Price == PrevSched of
					true ->
						ok;
					false ->
						error({price_not_prev_scheduled, H, Price, PrevSched})
				end,
				V2Price = maps:get(v2_price, CurrRow),
				case V2Price of
					error ->
						error({v2_price_missing_for_ema_expectation, H});
					_ ->
						ExpectedTargetPrice =
							case H < TransitionEnd of
								true ->
									Interval1 = H - TransitionStart,
									Interval2 = TransitionEnd - H,
									(TransitionStartPrice * Interval2 + V2Price * Interval1)
										div (Interval1 + Interval2);
								false ->
									V2Price
							end,
						EMAPrice = (9 * PrevSched + ExpectedTargetPrice) div 10,
						ExpectedSched = max(PrevSched div 2,
							min(PrevSched * 2, EMAPrice)),
						case Sched == ExpectedSched of
							true ->
								ok;
							false ->
								error({scheduled_price_mismatch, H,
									Sched, ExpectedSched,
									{expected_target, ExpectedTargetPrice},
									{ema, EMAPrice},
									{v2, V2Price},
									{prev_sched, PrevSched}})
						end
				end
		end
	end,
	Pairs),

AdjustHeights = [H || H <- Heights, H rem PriceAdjustFreq == 0],
true = (length(AdjustHeights) > 0),
io:format("  Block price fields consistent across ~p consecutive blocks  [OK]~n",
	[length(Rows)]),
io:format("  Recalculation verified at adjustment heights: ~p  [OK]~n",
	[AdjustHeights]),
ok.